# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902039


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/33 [00:00<?, ?it/s]

Rendering models:   6%|▌         | 2/33 [00:07<01:53,  3.68s/it]

Rendering models:   9%|▉         | 3/33 [00:07<01:19,  2.64s/it]

Rendering models:  12%|█▏        | 4/33 [00:07<00:56,  1.94s/it]

Rendering models:  15%|█▌        | 5/33 [00:07<00:38,  1.39s/it]

...

Rendering models:  88%|████████▊ | 29/33 [00:13<00:00,  5.02it/s]

Rendering models:  91%|█████████ | 30/33 [00:13<00:00,  4.65it/s]

Rendering models:  94%|█████████▍| 31/33 [00:13<00:00,  4.80it/s]

Rendering models:  97%|█████████▋| 32/33 [00:14<00:00,  4.52it/s]

ElisabethB                            0.000377
not-logged-in-cae133427d99aac1e554    0.000237
nickoftona                            0.000294
kayleebug2017                         0.001973
not-logged-in-aac1a549b28981db19c0    0.002393
tingard                               0.000188
BrownEyedGirl                         0.009803
Gorislavna                            0.001690
Poupoussin                            0.003910
Thedunkmasta                          0.003081
theetick                              0.003401
sn346808                              0.049337
sn346808                              0.022325
sn346808                              0.032018
not-logged-in-14d09f70263c1fe15cc3    0.001307
stellarastronomyisstellar             2.698752
ayluminati                            0.001974
Tenorclef                             0.002049
astronphoenix                         0.000228
not-logged-in-7b20cc95ac8c47c75645    0.000895
amidou                                0.002274
nthibert1    

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())